In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch
from sklearn.model_selection import train_test_split
import os

In [17]:
data = pd.read_csv('../data/combined/full_raw_data.csv')
data

,Unnamed: 0,GAME_SEQUENCE,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY_NAME,TEAM_WINS_LOSSES,PTS_QTR1,PTS_QTR2,PTS_QTR3,...,PTS_OT10,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,TOV,YEAR,GAME_DATE_EST
0,2012-10-30,1,21200001,1610612764,WAS,Washington,0-1,24,15,23,...,0.0,84,0.356,0.600,0.250,26,39,12,2012-2013,NaN
1,2012-10-30,1,21200001,1610612739,CLE,Cleveland,1-0,31,19,24,...,0.0,94,0.456,0.682,0.350,22,54,20,2012-2013,NaN
2,2012-10-30,2,21200002,1610612738,BOS,Boston,0-1,25,29,22,...,0.0,107,0.520,0.821,0.462,24,41,15,2012-2013,NaN
3,2012-10-30,2,21200002,1610612748,MIA,Miami,1-0,31,31,31,...,0.0,120,0.544,0.813,0.500,25,36,8,2012-2013,NaN
4,2012-10-30,3,21200003,1610612742,DAL,Dallas,1-0,25,23,26,...,0.0,99,0.471,0.778,0.333,22,40,11,2012-2013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17221,2019-04-10,9,21801228,1610612743,DEN,Denver,54-28,22,29,22,...,0.0,99,0.448,0.647,0.303,23,53,12,2018-2019,NaN
17222,2019-04-10,10,21801229,1610612762,UTA,Utah,50-32,26,40,31,...,0.0,137,0.443,0.879,0.400,31,57,17,2018-2019,NaN
17223,2019-04-10,10,21801229,1610612746,LAC,LA,48-34,39,31,32,...,0.0,143,0.509,0.767,0.429,34,52,12,2018-2019,NaN
17224,2019-04-10,11,21801230,1610612758,SAC,Sacramento,39-43,39,48,26,...,0.0,131,0.521,0.813,0.419,25,35,10,2018-2019,NaN


In [18]:
# rename Unnamed: 0 to Date
data = data.rename(columns={'Unnamed: 0': 'Date'})
data

,Date,GAME_SEQUENCE,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY_NAME,TEAM_WINS_LOSSES,PTS_QTR1,PTS_QTR2,PTS_QTR3,...,PTS_OT10,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,TOV,YEAR,GAME_DATE_EST
0,2012-10-30,1,21200001,1610612764,WAS,Washington,0-1,24,15,23,...,0.0,84,0.356,0.600,0.250,26,39,12,2012-2013,NaN
1,2012-10-30,1,21200001,1610612739,CLE,Cleveland,1-0,31,19,24,...,0.0,94,0.456,0.682,0.350,22,54,20,2012-2013,NaN
2,2012-10-30,2,21200002,1610612738,BOS,Boston,0-1,25,29,22,...,0.0,107,0.520,0.821,0.462,24,41,15,2012-2013,NaN
3,2012-10-30,2,21200002,1610612748,MIA,Miami,1-0,31,31,31,...,0.0,120,0.544,0.813,0.500,25,36,8,2012-2013,NaN
4,2012-10-30,3,21200003,1610612742,DAL,Dallas,1-0,25,23,26,...,0.0,99,0.471,0.778,0.333,22,40,11,2012-2013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17221,2019-04-10,9,21801228,1610612743,DEN,Denver,54-28,22,29,22,...,0.0,99,0.448,0.647,0.303,23,53,12,2018-2019,NaN
17222,2019-04-10,10,21801229,1610612762,UTA,Utah,50-32,26,40,31,...,0.0,137,0.443,0.879,0.400,31,57,17,2018-2019,NaN
17223,2019-04-10,10,21801229,1610612746,LAC,LA,48-34,39,31,32,...,0.0,143,0.509,0.767,0.429,34,52,12,2018-2019,NaN
17224,2019-04-10,11,21801230,1610612758,SAC,Sacramento,39-43,39,48,26,...,0.0,131,0.521,0.813,0.419,25,35,10,2018-2019,NaN


In [19]:
# write to csv
data.to_csv('../data/combined/full_raw_data.csv', index=False)